In [281]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder , OneHotEncoder ,OrdinalEncoder, TargetEncoder, StandardScaler , RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [282]:
df = pd.read_csv("Data_Train.csv")

In [283]:
df=df.drop(columns=('Additional_Info'))

In [284]:
df=df.dropna()

In [285]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10682 entries, 0 to 10682
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10682 non-null  object
 1   Date_of_Journey  10682 non-null  object
 2   Source           10682 non-null  object
 3   Destination      10682 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10682 non-null  object
 6   Arrival_Time     10682 non-null  object
 7   Duration         10682 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Price            10682 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 918.0+ KB


In [286]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,13302


In [287]:
df.shape # rows and columns

(10682, 10)

In [288]:
x=df.drop("Price",axis=1)
y=df.Price

In [289]:
xtrain,xtest,ytrain,ytest= train_test_split(x,y,train_size=0.8,random_state=42)

In [290]:
num_cols = xtrain.select_dtypes(include='number').columns
cat_cols = xtrain.select_dtypes(exclude='number').columns

In [291]:
def cat_cols_nunique(cols):
    for col in cols:
        print(f'{col} - {df[col].nunique()}')

cat_cols_nunique(cat_cols)

Airline - 12
Date_of_Journey - 44
Source - 5
Destination - 6
Route - 128
Dep_Time - 222
Arrival_Time - 1343
Duration - 368
Total_Stops - 5


In [292]:
df["Route"].unique()

array(['BLR ? DEL', 'CCU ? IXR ? BBI ? BLR', 'DEL ? LKO ? BOM ? COK',
       'CCU ? NAG ? BLR', 'BLR ? NAG ? DEL', 'CCU ? BLR',
       'BLR ? BOM ? DEL', 'DEL ? BOM ? COK', 'DEL ? BLR ? COK',
       'MAA ? CCU', 'CCU ? BOM ? BLR', 'DEL ? AMD ? BOM ? COK',
       'DEL ? PNQ ? COK', 'DEL ? CCU ? BOM ? COK', 'BLR ? COK ? DEL',
       'DEL ? IDR ? BOM ? COK', 'DEL ? LKO ? COK',
       'CCU ? GAU ? DEL ? BLR', 'DEL ? NAG ? BOM ? COK',
       'CCU ? MAA ? BLR', 'DEL ? HYD ? COK', 'CCU ? HYD ? BLR',
       'DEL ? COK', 'CCU ? DEL ? BLR', 'BLR ? BOM ? AMD ? DEL',
       'BOM ? DEL ? HYD', 'DEL ? MAA ? COK', 'BOM ? HYD',
       'DEL ? BHO ? BOM ? COK', 'DEL ? JAI ? BOM ? COK',
       'DEL ? ATQ ? BOM ? COK', 'DEL ? JDH ? BOM ? COK',
       'CCU ? BBI ? BOM ? BLR', 'BLR ? MAA ? DEL',
       'DEL ? GOI ? BOM ? COK', 'DEL ? BDQ ? BOM ? COK',
       'CCU ? JAI ? BOM ? BLR', 'CCU ? BBI ? BLR', 'BLR ? HYD ? DEL',
       'DEL ? TRV ? COK', 'CCU ? IXR ? DEL ? BLR',
       'DEL ? IXU ? BOM ? COK', 'CCU 

>Date

>Preprocessing Total_Stops

In [293]:
df["Total_Stops"]

0        non-stop
1         2 stops
2         2 stops
3          1 stop
4          1 stop
           ...   
10678    non-stop
10679    non-stop
10680    non-stop
10681    non-stop
10682     2 stops
Name: Total_Stops, Length: 10682, dtype: object

In [294]:
# Converting Total_Stops: 'non-stop' -> 0, remove 'stop(s)' and cast to int

if 'Total_Stops' in df.columns:
    df['Total_Stops'] = df['Total_Stops'].astype(str)
    print('Total_Stops unique before:', df['Total_Stops'].unique())

    # replacing non-stop with 0, then extract digits from values like '1 stop' or '2 stops'
    df['Total_Stops'] = df['Total_Stops'].str.replace('non-stop', '0', case=False, regex=False)
    df['Total_Stops'] = df['Total_Stops'].str.replace('1 stop', '1', case=False, regex=False)
    df['Total_Stops'] = df['Total_Stops'].str.replace('2 stops', '2', case=False, regex=False)
    df['Total_Stops'] = df['Total_Stops'].str.replace('3 stops', '3', case=False, regex=False)
    df['Total_Stops'] = df['Total_Stops'].str.replace('4 stops', '4', case=False, regex=False)
    df['Total_Stops'] = df['Total_Stops'].astype(int)
    print('Total_Stops unique after:', df['Total_Stops'].unique())
else:
    print('Column Total_Stops not found in dataframe')

Total_Stops unique before: ['non-stop' '2 stops' '1 stop' '3 stops' '4 stops']
Total_Stops unique after: [0 2 1 3 4]


>Dep_Time

In [295]:
# Converted dept_time into minutes
df['Dep_Time'] = pd.to_datetime(df['Dep_Time'])
df['Dep_Hour'] = df['Dep_Time'].dt.hour
df['Dep_Minute'] = df['Dep_Time'].dt.minute


# df["Dep_Time"]

C:\Users\Mahammad Furkhan\AppData\Local\Temp\ipykernel_2436\1367562764.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dep_Time'] = pd.to_datetime(df['Dep_Time'])


>Arrival_Time

In [296]:
df['Arrival_Time'] = pd.to_datetime(df['Arrival_Time'])
df['Arrival_Hour'] = df['Arrival_Time'].dt.hour
df['Arrival_Minute'] = df['Arrival_Time'].dt.minute

C:\Users\Mahammad Furkhan\AppData\Local\Temp\ipykernel_2436\818625300.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Arrival_Time'] = pd.to_datetime(df['Arrival_Time'])


>Duration

In [297]:
def convert_duration(duration):
    hours = 0
    minutes=0

    if 'h' in duration:
        hours = int(duration.split('h')[0])
    if 'm' in duration:
        minutes = int(duration.split('m')[0].split()[-1])

    return hours * 60 + minutes
df["Duration"] = df["Duration"].apply(convert_duration)
df["Duration"]

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Name: Duration, Length: 10682, dtype: int64

In [298]:
num_cols = xtrain.select_dtypes(include='number').columns
cat_cols = xtrain.select_dtypes(exclude='number').columns

In [299]:
# Preprocessing transformer
preprocessor = ColumnTransformer(
        transformers=[
            ('Scaler', RobustScaler(), num_cols),
            ('Encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_cols),
        ],
        remainder='drop'
    )

In [ ]:
pipeline = Pipeline([
        ('preprocessing', preprocessor),
        ('model', RandomForestRegressor(criterion="absolute_error",n_estimators=200, random_state=42, max_depth=15, min_samples_leaf=5, min_samples_split=10))
    ])

In [ ]:
# pipeline.fit(xtrain, ytrain)

In [ ]:
train_score=pipeline.score(xtrain,ytrain)
train_score

0.953447687234622

In [ ]:
test_score=pipeline.score(xtest,ytest)
test_score

0.8342577222106065